In [6]:
import torch
from torch.nn.functional import one_hot
import torch_explain as te
from torch_explain.nn.functional import l1_loss
from torch_explain.logic.nn import psi
from torch_explain.logic.metrics import test_explanation , complexity


x0 = torch.zeros((4, 100))
x_train = torch.tensor([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
], dtype=torch.float)
x_train = torch.cat([x_train, x0], dim=1)
y_train = torch.tensor([1, 0, 0, 1], dtype=torch.long)


In [7]:
layers = [
    te.nn.EntropyLinear(x_train.shape[1], 10, n_classes=2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(10, 4),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(4, 1),
]
model = torch.nn.Sequential(*layers)

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
loss_form = torch.nn.CrossEntropyLoss()
model.train()
for epoch in range(1001):
    optimizer.zero_grad()
    y_pred = model(x_train).squeeze(-1)
    loss = loss_form(y_pred, y_train)
    print(y_pred, y_train)
    loss = loss_form(y_pred, y_train) + 0.00001 * te.nn.functional.entropy_logic_loss(model)
    loss.backward()
    optimizer.step()

tensor([[-7.8426,  2.5257],
        [ 6.6743, -2.0974],
        [ 4.3273, -5.4073],
        [-8.3164,  1.0471]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[-7.8746,  2.6461],
        [ 6.9059, -1.9832],
        [ 0.2643, -2.5603],
        [-3.4180,  0.2600]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[-7.8010,  2.5540],
        [ 6.8139, -2.1409],
        [ 3.4384, -4.8085],
        [-6.9346,  0.7705]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[-7.7580,  2.4643],
        [ 5.4371, -2.0295],
        [ 4.6608, -5.7361],
        [-6.8620,  0.1744]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[-7.7226,  2.3951],
        [ 3.2896, -1.0468],
        [ 4.7379, -5.8850],
        [-5.0090, -0.8506]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[-7.6527,  2.3432],
        [ 2.6805, -0.8993],
        [ 4.7593, -5.9761],
        [-4.4701, -1.1502]], grad_fn=<SqueezeBackward1>) tensor([1, 0, 0, 1])
tensor([[-7.5612,  2.3062],


In [5]:
from torch_explain.logic.nn import entropy
from torch.nn.functional import one_hot
y1h = one_hot(y_train)
explanation, _ = entropy.explain_class(model, x_train, y1h, x_train, y1h, target_class=1)

print(model(x_train[0]))

print(explanation)

tensor([[[-7.2057],
         [ 0.8409]]], grad_fn=<AddBackward0>)
(feature0000000000 & feature0000000001) | (~feature0000000000 & ~feature0000000001)


In [5]:
from torch_explain.logic.metrics import test_explanation, complexity
accuracy, preds = test_explanation(explanation, x_train, y1h, target_class=1)
explanation_complexity = complexity(explanation)

print(explanation_complexity, accuracy, preds)

4 1.0 [ True False False  True]
